# Cálculo de métricas de los cuestionarios

El objetivo de este notebook será calcular con los datos obtenidos de las entregas de cuestionarios por parte de los alumnos las siguientes métricas:

- Número de intentos totales a lo largo del curso
- Número de intentos por mes
- Número máximo de días consecutivos  sin ningúna entrega
- Promedio de intentos por cuestionario

## Configuración inicial


In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import count
from itertools import count
from pyspark.sql.functions import count as spark_count
from pyspark.sql.functions import from_unixtime, date_format
import os

ruta_origen = "/home/carlos/Documentos/TFG/spark-workspace/data/raw/ip"
ruta_destino = "/home/carlos/Documentos/TFG/spark-workspace/data/metrics"
os.makedirs(ruta_destino, exist_ok=True)

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Creacion de metricas de los cuestionarios") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

## Carga de datos

In [41]:
quiz_df = spark.read.parquet(f"{ruta_origen}/quiz_attempts_st_ip.parquet")
quiz_df.printSchema()

root
 |-- userid: string (nullable = true)
 |-- quiz_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timeopen: long (nullable = true)
 |-- timeclose: long (nullable = true)
 |-- state: string (nullable = true)
 |-- attempt: long (nullable = true)
 |-- sumgrades: string (nullable = true)
 |-- timestart: long (nullable = true)
 |-- timefinish: long (nullable = true)



## Número de  intentos totales a lo largo del curso

In [32]:

# Agrupar por 'userid' y contar el número de intentos
num_total_attempts = quiz_df.groupBy("userid").agg(spark_count("*").alias("quiz_total_submits"))
num_total_attempts.show(300, truncate=False)

num_total_attempts.write.mode("overwrite").parquet(f"{ruta_destino}/quiz_total_submits.parquet")

+----------------------------------------------------------------+------------------+
|userid                                                          |quiz_total_submits|
+----------------------------------------------------------------+------------------+
|c38c8db0d56175f74d03a010b407bdb88b404779e64735f323c1d5d90b6629ed|15                |
|484eda6cc4b97c95231f680fd082d8285d16c4712419d0640ab9ee52cacd389c|5                 |
|61e02718ca4728cba587fccbae7cc24fb59ab10c6d8aeeddc21e14a862e6829e|10                |
|cf900f2b0c3fbbe1883957602cf867549c88a80384ed66573fb5abb5be156582|4                 |
|3b8d431cbee3182d06225f9d5ab51f5806e8042f145e0ad0e37d98f56ae78f3d|11                |
|637568533d1cea5abb5fc41907d7ad146fbf0cf1aefc3a58d6eb95e85ca1c5e5|13                |
|49206beba70f0e7d7fabf787c5565196c3eed3915b8a7fda6a84e26310b11db8|15                |
|b800f658ee51bbcc32ffe8b8e2b504643b4a00e393737f8b7b6ed699601b65f5|10                |
|58f92ea7e6ad27de174bde72af7a6574bc5f8fef62ee6babdc633

## Número de intentos por mes

Para conseguir esta métrica, en primer lugar se van a añadir dos nuevas columnas al dataframe que indiquen el año de realización del cuestionario y el mes. Una vez llevado a cabo esto, se filtrará para quedarnos solo con los intentos hechos en 2023, y posteriormente agruparemos por usuario y mes para contar el número de entregas hechas por usuario en cada mes.

En último lugar, se agrupará por usuario y se pivotará la columna de meses para llevarnos cada mes entre septiembre y diciembre a una columna en la que aparecerá el número de intentos hecho por usuario en cada mes.

In [33]:

#Añadimos las columnas de mes y año de envío del cuestionario
df_monthYear = quiz_df.withColumn("month_submit", date_format(from_unixtime(col("timestart")), "MM")) \
      .withColumn("year_submit", date_format(from_unixtime(col("timestart")), "yyyy"))

df_monthYear.show(5, truncate=False)

#Filtramos por año 2023 y comprobamos el número de intentos antes y después del filtrado
print("Número de intentos antes de filtrar por año:", df_monthYear.count())

df_monthYear_2023 = df_monthYear.filter(col("year_submit") == "2023")

print("Número de intentos tras filtrar por año 2023:", df_monthYear_2023.count())


+----------------------------------------------------------------+-------+--------------------------------+----------+---------+--------+-------+---------+----------+----------+------------+-----------+
|userid                                                          |quiz_id|name                            |timeopen  |timeclose|state   |attempt|sumgrades|timestart |timefinish|month_submit|year_submit|
+----------------------------------------------------------------+-------+--------------------------------+----------+---------+--------+-------+---------+----------+----------+------------+-----------+
|5220ebcc6c2637b854099b7c7658e3b3ec8a601fb6ec1bb12e7dd6deda800e7f|16681  |Cuestionario previo a la clase 1|1694507400|0        |finished|1      |3.00000  |1694508376|1694535081|09          |2023       |
|e6d549bd33db7e6e9cacf7a4030245ca8d8f24184df4335a102828be221659c6|16681  |Cuestionario previo a la clase 1|1694507400|0        |finished|1      |4.00000  |1694508380|1694527333|09         

Podemos comprobar que prácticamente es igual el número de intentos antes y después de filtrar por el año 2023, así que es claro que todos los intentos prácticamente tuvieron lugar durante ese periodo, al ser el periodo lectivo de la asignatura.

Vamos a comenzar a agrupar para calcular las métricas por mes para cada usuario.

In [34]:
submits_per_month= df_monthYear_2023.groupBy("userid", "month_submit") \
      .agg(spark_count("*").alias("num_submits")).orderBy("userid", "month_submit")
      
submits_per_month.show(300, truncate=False) 

+----------------------------------------------------------------+------------+-----------+
|userid                                                          |month_submit|num_submits|
+----------------------------------------------------------------+------------+-----------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|09          |6          |
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|10          |1          |
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|12          |1          |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|09          |6          |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|10          |3          |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|11          |4          |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|09          |6          |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|10          |3

Tras agruparlas, ahora podemos proceder a pivotar los meses para quedar con un formato en el cual haya una tupla por usuario y columnas para el número de entregas para cada mes

In [35]:
from pyspark.sql.functions import first

submits_per_month_pivoted = submits_per_month.groupBy("userid") \
      .pivot("month_submit", ["09", "10", "11", "12"]).agg(first("num_submits")).orderBy("userid")
      
submits_per_month_pivoted.show(300, truncate=False)

+----------------------------------------------------------------+----+----+----+----+
|userid                                                          |09  |10  |11  |12  |
+----------------------------------------------------------------+----+----+----+----+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|6   |1   |NULL|1   |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|6   |3   |4   |NULL|
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|6   |3   |4   |NULL|
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|11  |6   |5   |NULL|
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|2   |NULL|1   |NULL|
|091af124e119a447c7f6594fb2f7c4fbb678f669966db01e3f62c26eedb220af|5   |1   |2   |4   |
|0a2e27fd5eb3547b064f5bcd8a26472c8802e19a5c158b380539e85b42d194d9|8   |4   |2   |NULL|
|11c0d56ee71665bfe766f1a57c333061cb34d747204264191ee23b775e6edab4|7   |4   |4   |NULL|
|13a777f8c88ba748246449dfb45dcb8f76056a22f0

Una vez obtenidas las métricas deseadas, solo haremos un pequeño cambio de nombre a las columnas y escribiremos  el dataframe como parquet

In [37]:
submits_per_month_pivoted = submits_per_month_pivoted \
      .withColumnRenamed("09", "quizes_submitted_firstMonth") \
      .withColumnRenamed("10", "quizes_submitted_secondMonth") \
      .withColumnRenamed("11", "quizes_submitted_thirdMonth") \
      .withColumnRenamed("12", "quizes_submitted_fourthMonth")
      
submits_per_month_pivoted.show(5, truncate=False)

submits_per_month_pivoted.write.mode("overwrite").parquet(f"{ruta_destino}/quiz_submits_per_month.parquet")

+----------------------------------------------------------------+---------------------------+----------------------------+---------------------------+----------------------------+
|userid                                                          |quizes_submitted_firstMonth|quizes_submitted_secondMonth|quizes_submitted_thirdMonth|quizes_submitted_fourthMonth|
+----------------------------------------------------------------+---------------------------+----------------------------+---------------------------+----------------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|6                          |1                           |NULL                       |1                           |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|6                          |3                           |4                          |NULL                        |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|6                          |3

## Número máximo de dias consecutivos sin ninguna entrega

Para calcular esta métrica, en primer lugar añadiremos una columna al dataframe base utilizado en el cálculo de la anterior métrica que indique el día de realización de una tarea convirtiendolo a  un entero, y las ordenaremos por fecha.   

Tras llevar a cabo esto, agruparemos el dataframe por usuario, quedándonos para cada uno con una lista con los datos temporales de cada una de sus entregas.

Después, definiremos una función que será envuelta en una UDF que nos permita obtener el máximo número de días que ha pasado cada estudiante sin realizar una tarea cuando la apliquemos sobre el anterior dataframe ya agrupado.

El resultado será un df con la métrica calculada para cada estudiante, el cual escribiremos a parquet.

### Añadir columna día, filtrar y ordenar

In [ ]:
#Obtenemos el día de la fecha de inicio del cuestionario
df_monthYearDay=df_monthYear.withColumn("day_submit", date_format(from_unixtime(col("timestart")), "dd")).orderBy("timestart")

#Lo convertimos a entero
df_monthYearDay = df_monthYearDay.withColumn("day_submit", col("day_submit").cast("int"))
print("Número de entregas antes de quedarnos solo con las que fueron terminadas: ", df_monthYearDay.count())

#Nos quedamos solo con las entregas cuyo estado sea "finished"
df_monthYearDay = df_monthYearDay.filter(col("state") == "finished")
print("Número de entregas tras  quedarnos solo con las que fueron terminadas: ", df_monthYearDay.count())

#Ordenamos el dataframe por id de usuario y por fecha de realización de la tarea
df_monthYearDay = df_monthYearDay.orderBy("userid", "timestart")

df_monthYearDay.printSchema()
df_monthYearDay.show(300, truncate = False)

Número de entregas antes de quedarnos solo con las que fueron terminadas:  2414
Número de entregas tras  quedarnos solo con las que fueron terminadas:  2391
root
 |-- userid: string (nullable = true)
 |-- quiz_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timeopen: long (nullable = true)
 |-- timeclose: long (nullable = true)
 |-- state: string (nullable = true)
 |-- attempt: long (nullable = true)
 |-- sumgrades: string (nullable = true)
 |-- timestart: long (nullable = true)
 |-- timefinish: long (nullable = true)
 |-- month_submit: string (nullable = true)
 |-- year_submit: string (nullable = true)
 |-- day_submit: integer (nullable = true)

+----------------------------------------------------------------+-------+-------------------------------------------------------------------+----------+----------+--------+-------+---------+----------+----------+------------+-----------+----------+
|userid                                                          |quiz_id|n

### Agrupar datos de las entregas hechas por cada usuario

In [53]:
from  pyspark.sql.functions import collect_list, struct
df_entregas_usuario = df_monthYearDay.groupBy("userid").agg(collect_list(struct("month_submit", "day_submit")).alias("entregas"))

df_entregas_usuario.show(5, truncate=False)

+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userid                                                          |entregas                                                                                                                                                                                                      |
+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|[{09, 12}, {09, 13}, {09, 18}, {09, 20}, {09, 25}, {09, 25}, {10, 3}, {12, 1}]                                  

### Definir la función que calcula el máximo número de días sin conexiones


In [62]:
import datetime

def max_dias_sin_entregar(entregas):
    """
    Calcula el máximo número de días consecutivos sin entregar un cuestionario.
    :param entregas: Lista de Rows con month_submit (str) y day_submit (int)
    :return: int
    """
    if len(entregas) < 2:
        return 0

    # Convertir cada entrega a fecha real
    fechas = []
    for e in entregas:
        mes = int(e.month_submit)
        dia = int(e.day_submit)
        try:
            fecha = datetime.date(2023, mes, dia)  # Año fijo para todas
            fechas.append(fecha)
        except ValueError:
            continue  # Saltar fechas mal formadas

    fechas.sort()
    max_diff = 0

    for i in range(len(fechas) - 1):
        diff = (fechas[i + 1] - fechas[i]).days
        if diff > max_diff:
            max_diff = diff

    return max_diff - 1  # Quitamos el día de la siguiente entrega


### Registrarla como UDF 

In [63]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

max_dias_udf = udf(max_dias_sin_entregar, IntegerType())


### Aplicar el UDF al dataframe y ver resultado

In [ ]:

df_max_dias_sin_entrega = df_entregas_usuario.withColumn("max_dias_sin_entregar", max_dias_udf("entregas"))
df_max_dias_sin_entrega.show(300, truncate = False)

+----------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+
|userid                                                          |entregas                                                                                                                                                                                                                                                                                                                                                                                                                 |max_dias_sin_entregar|
+-

### Escribimos el resultado de la métrica calculada

In [69]:
# Quitamos la columna que tiene la lista de entregas
df_max_dias_sin_entrega = df_max_dias_sin_entrega.drop("entregas").withColumnRenamed("max_dias_sin_entregar", "quiz_max_days_without_submit")
df_max_dias_sin_entrega.show(350, truncate=False)
submits_per_month_pivoted.write.mode("overwrite").parquet(f"{ruta_destino}/quiz_max_days_without_submit.parquet")

+----------------------------------------------------------------+----------------------------+
|userid                                                          |quiz_max_days_without_submit|
+----------------------------------------------------------------+----------------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|58                          |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|14                          |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|13                          |
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|14                          |
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|49                          |
|091af124e119a447c7f6594fb2f7c4fbb678f669966db01e3f62c26eedb220af|39                          |
|0a2e27fd5eb3547b064f5bcd8a26472c8802e19a5c158b380539e85b42d194d9|32                          |
|11c0d56ee71665bfe766f1a57c333061cb34d74